# Demo

This notebook was made for a live demo. The demo dataset is a copy of the original one, but with a few games I played against a CPU added. I would play for a bit (~30 seconds to ensure at least a few sequences long enough to predict on) and then run the pretrained model on my inputs. This is not a great representation of the model's actual performance in my original testing for a variety of reasons, but it is fun!

In [ ]:
import os
import pandas as pd
import numpy as np

# py-slippi
from slippi import Game

In [ ]:
PATH='dataset/demo/inference'
SEQ_LEN=180

In [ ]:
id_to_player = {
    0: '2saint',
    1: 'aklo',
    2: 'amsa',
    3: 'axe',
    4: 'cody',
    5: 'cpu', # The computer that I played against
    6: 'cynthia', # That's me!
    7: 'hbox',
    8: 'kodorin',
    9: 'logan',
    10: 'mang0',
    11: 'moky',
    12: 'n0ne',
    13: 'plup',
    14: 'sfat',
    15: 'wizzrobe',
    16: 'yingling',
    17: 'zain'
}

## Load the game

In [ ]:
file = os.path.join(PATH, os.listdir(PATH)[0])
game = Game(file)
game

## Get the inputs from the game

These functions can be found in [inputs.py](inputs.py), but I wanted to talk about them a bit in my demo, so they are also here.

In [ ]:
# get the port indices used by the players
def getPorts(game):
    ports = []
    for i in range(0, len(game.start.players)):
        if game.start.players[i] is not None:
            ports.append(i)

    return ports

# get the characters for each player, just for fun
def getCharacters(game):
    characters = []
    for player in game.start.players:
        if player is not None:
            characters.append(player.character)
    
    return characters

In [ ]:
getCharacters(game)

In [ ]:
START_FRAME = 64 # game starts on frame 64
COLS = ['joy_x', 'joy_y', 'cstick_x', 'cstick_y', 'z', 'r_dig', 'l_dig', 'a', 'b', 'x', 'y']
BUTTONS = ['Z', 'R', 'L', 'A', 'B', 'X', 'Y'] # names of the buttons in py-slippi

def getFrameInputs(player):
    # analog stick / c stick
    analog = [player.joystick.x, player.joystick.y, player.cstick.x, player.cstick.y]

    # buttons
    pressed_buttons = []
    # get the names of the buttons currently being pressed
    logical_pressed_names = map(lambda x: x.name, player.buttons.physical.pressed())

    for b in BUTTONS:

        if b in logical_pressed_names:
            pressed_buttons.append(1)
        else:
            pressed_buttons.append(0)

    return analog + pressed_buttons 

def getGameInputs(game):
    p1_inputs = pd.DataFrame(columns=COLS)
    p2_inputs = pd.DataFrame(columns=COLS)

    # get the controller port #s of the players
    p1_port, p2_port = getPorts(game)

    for i in range(START_FRAME, len(game.frames)):
        p1 = game.frames[i].ports[p1_port].leader.pre 
        p1_inputs.loc[len(p1_inputs.index)] = getFrameInputs(p1)

        p2 = game.frames[i].ports[p2_port].leader.pre
        p2_inputs.loc[len(p2_inputs.index)] = getFrameInputs(p2)

    return p1_inputs, p2_inputs

In [ ]:
p1_inputs, _ = getGameInputs(game)
p1_inputs.head(10)

In [ ]:
p1_inputs.shape

## Compute the displacement vectors

In [ ]:
def nextDisplacement(df, index):
    velocities = [] # stores both x and y displacement
    for col in df.columns:
        velocities.append(df.at[index+1, col] - df.at[index, col])

    return velocities

def gameDisplacements(game):
    ds = []
    prev_frame = 0
    for i in range(game.shape[0] - 1):
        # get next displacement
        d = nextDisplacement(game, i)

        if not d == [0, 0]: # only include frames that have some velocity
            # get elapsed
            elapsed = i - prev_frame
            prev_frame = i

            if elapsed > 60: # arbitrarily set all values > 60 to 0
                elapsed = 0

            ds.append((d[0], d[1], elapsed))

    return pd.DataFrame(ds, columns=['joy_x', 'joy_y', 'elapsed'])

In [ ]:
displacements = gameDisplacements(p1_inputs[['joy_x', 'joy_y']])

In [ ]:
displacements.head(10)

In [ ]:
displacements.shape

## Scale elapsed values

In [ ]:
max = 60 # 60 is the maximum value allowed in the training data, so we scale based on that

normed = displacements.copy()
normed['elapsed'] = normed['elapsed'].map(lambda x : x / max)

In [ ]:
displacements.head(10)

## Process into sequences

In [ ]:
def seqs_from_df(df, seq_len=SEQ_LEN):
    """
    Arrange a dataframe of inputs into sequences of inputs of length seq_len.
    """
    seqs = []

    # set the slide factor to 0.5 instead of the 
    for i in range(0, df.shape[0] - seq_len, int(seq_len*0.5)):
        seq_x = df.loc[i:i+seq_len-1]
        seqs.append(np.array(seq_x, dtype=np.float32))
        
    return seqs

In [ ]:
sequences = seqs_from_df(normed)

print(f'{len(sequences)=}')
sequences[0][0:10]

## Load the pre-trained model

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, ELU, Dropout, Dense, Concatenate, LSTM

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      # allocated memory as needed, rather than 100% of it all the time
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
def createClassifier(width=3, seq_len=180):
    input_layer = Input(shape=(seq_len, width))
    conv1 = Conv1D(filters=32, kernel_size=7, strides=2, activation=ELU())(input_layer)
    conv2 = Conv1D(filters=32, kernel_size=3, strides=1, activation=ELU())(input_layer)

    catted = Concatenate(axis=1)([conv1, conv2])
    elu1 = ELU(32)(catted)
    conv3 = Conv1D(filters=32, kernel_size=2, strides=1, activation=ELU())(elu1)
    conv4 = Conv1D(filters=32, kernel_size=2, strides=1, activation=ELU())(conv3)
    drop1 = Dropout(0.2)(conv4)

    gru1 = LSTM(32, return_sequences=True)(drop1)
    gru2 = LSTM(32)(gru1)
    drop2 = Dropout(0.2)(gru2)

    output = Dense(18, activation='softmax')(drop2)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = createClassifier(3, 180)

model.load_weights('models/demo')
model.summary()

## Predict on the new data

In [ ]:
predicts = model.predict(np.array(sequences, dtype=np.float32))

hard_predicts = [np.argmax(predicts[i]) for i in range(predicts.shape[0])]

[id_to_player[hp] for hp in hard_predicts]

Thank God that worked. Or sorry that it didn't idk